## ICESat-2 and Reference Elevation Comparison

This notebook compares ICESat-2 elevations to reference elevations calculted from DTM_reference_elevations_calculation_sliderulecsv.m or DTM_reference_elevations_calculation.m

